In [0]:
# this is the sane notebook as stocks_load_explore_nb
# except it plugs in functions from stocks_functions_helper
# so the functions helper notebook is run
# and we have a streamlined workflow - to execute its functions on 
# on the futures and options data location

In [0]:
%run  "./stocks_functions_helper"

In [0]:
fo_data_location = '/mnt/fopart'
fo_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv(fo_data_location)

In [0]:
fodf_grouped_instrument = group_df(fo_df).cache()

In [0]:
fodf_grouped_instrument.show(2)

+-------------------+------+----------+-----------+----------+----+-----+---+---------+-----+--------+
 rts|SYMBOL|INSTRUMENT| EXPIRY_DT|OPTION_TYP|YEAR|MONTH|DAY|Contracts|VALUE|OPEN_INT|
+-------------------+------+----------+-----------+----------+----+-----+---+---------+-----+--------+
2019-08-28 00:00:00| ACC| OPTSTK|31-Oct-2019| CE|2019| 8| 28| 0.0| 0.0| 0.0|
2019-08-28 00:00:00| ACC| OPTSTK|31-Oct-2019| PE|2019| 8| 28| 0.0| 0.0| 0.0|
+-------------------+------+----------+-----------+----------+----+-----+---+---------+-----+--------+
only showing top 2 rows

In [0]:
fodf_pcr = add_pcr_to_df(fodf_grouped_instrument).cache()

In [0]:
fodf_pcr.show(2)

+-------------------+----------+-----------+----+-----+---+----+----+----+----+
 rts| SYMBOL| EXPIRY_DT|YEAR|MONTH|DAY| XX| PE| CE| pcr|
+-------------------+----------+-----------+----+-----+---+----+----+----+----+
2019-08-28 00:00:00| ACC|31-Oct-2019|2019| 8| 28|null| 0.0| 0.0|null|
2019-08-28 00:00:00|TORNTPOWER|26-Sep-2019|2019| 8| 28|null|25.0|20.0|1.25|
+-------------------+----------+-----------+----+-----+---+----+----+----+----+
only showing top 2 rows

In [0]:
fodf_processed = combine_grouped_and_pcr_dfs(
    fodf_grouped_instrument,fodf_pcr).cache()

In [0]:
fodf_processed.show(2)

+-------------------+------+-----------+----------+----------+---------+-----+--------+----+----+-----+---+
 rts|SYMBOL| EXPIRY_DT|INSTRUMENT|OPTION_TYP|CONTRACTS|VALUE|OPEN_INT| PCR|YEAR|MONTH|DAY|
+-------------------+------+-----------+----------+----------+---------+-----+--------+----+----+-----+---+
2019-08-28 00:00:00| ACC|31-Oct-2019| OPTSTK| CE| 0.0| 0.0| 0.0|null|2019| 8| 28|
2019-08-28 00:00:00| ACC|31-Oct-2019| OPTSTK| PE| 0.0| 0.0| 0.0|null|2019| 8| 28|
+-------------------+------+-----------+----------+----------+---------+-----+--------+----+----+-----+---+
only showing top 2 rows

In [0]:
write_processed_output(fodf_processed,'/mnt/foprocessed', 'overwrite', 'FOPR')